In [1]:
import pandas as pd
import re
import numpy as np
import nltk
from collections import Counter
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.cm as cmap
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
#from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.stem import PorterStemmer
from nltk import word_tokenize,sent_tokenize
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
#from nltk.corpus import stopwords
#stop = stopwords.words('english')
from sklearn.cross_validation import train_test_split


/app_2/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("./train_2kmZucJ.csv")
test = pd.read_csv("./test_oJQbWVk.csv")


In [26]:

X_train, X_val, y_train, y_val = train_test_split(train['tweet'].values, 
                 train['label'].values,        
                 test_size=0.2)

X_train = X_train.tolist()
X_train



['Miss Anaheim beauty pageant. #samsung #note3 @samsungusa #beautiful #instamood #jagagency... http://fb.me/1r4jGh3Z3',
 '@guy_interruptd im on my 6th goddam iphone, and now its giving trouble!! Grrr!!! My n95 was great until i drove over it',
 'My phone is alive :) after he decided to go for a swim on Saturday, spend three days in a bowl of rice...and its alive. #Motorola',
 '@samsungus & @apple are preparing to release a #foldable phone @motorolaus has filed a patten for a self repairing screen function. . . #apple #influencer #samsunggalaxy #tech #photooftheday #money #samsung #save #gadgets #tryit2 #technology #motorola #in',
 'This is the #Book about which there is no doubt, #Iphone http://apple.co/1d3DZux #Android http://ift.tt/OQUrSI #rt #Usa #follow',
 'Welcome to the #family #future #samsung #s9 #consultancy #ilovemyjob #bethefirst\xe2\x80\xa6 https://www.instagram.com/p/BgYYfPfDFpk/',
 'Workin towards this Aston #astonmartin #tweegram #movies #instagood #iphone #android #inst

In [27]:
def data_cleaning(tweet):
    abc = []
    for i in range(0,len(tweet)):
        tweetx = tweet[i]
        tweetx = re.sub(r'\$\w*','',tweetx) # Remove tickers
        tweetx = re.sub(r'https?:\/\/.*\/\w*','',tweetx) # Remove hyperlinks
        tweetx = re.sub(r'['+string.punctuation+']+', ' ',tweetx) # Remove puncutations like 's
        #stop = set(stopwords.words('english'))
        tweetx = re.sub(r'[^a-zA-Z0-9]'," ",tweetx)
        stop_words=set(['a',    'about', 'above', 'after',   'again',  'against',              'ain',      'all',        'am',               'an',       'and',     'any',     'are',      'as',        'at',        'be',       'because',            'been',   'before',               'being',  'below', 'between',           'both',   'but',      'by',        'can',     'couldn',               'd',               'did',      'didn',    'do',       'does',   'doesn', 'doing',  'don',     'down',  'during',               'each',               'few',     'for',      'from',   'further',              'had',     'hadn',   'has',      'hasn',   'have',   'haven',               'having',               'he',       'her',      'here',    'hers',    'herself',              'him',     'himself',               'his',       'how',    'i',           'if',         'in',         'into',     'is',         'isn',       'it',         'its',        'itself',               'just',     'll',          'm',        'ma',      'me',      'mightn',              'more',  'most',   'mustn', 'my',               'myself',               'needn', 'now',    'o',         'of',        'off',      'on',       'once',   'only',    'or',               'other',  'our',      'ours',    'ourselves',          'out',      'over',    'own',    're',        's',          'same',               'shan',   'she',      'should',               'so',        'some',  'such',    't',          'than',    'that',    'the',               'their',   'theirs',  'them',  'themselves',      'then',    'there',  'these',  'they',    'this',     'those',               'through',            'to',        'too',     'under', 'until',    'up',       've',        'very',    'was',     'we',               'were',   'weren',               'what',   'when',  'where',               'which', 'while',  'who',    'whom',               'why',    'will',      'with',    'won',    'y',          'you',     'your',    'yours',  'yourself',               'yourselves'])
        exclude = set(string.punctuation)
        #exclude1= set(custom_list)
        stop_words.update(exclude)
        lemma = WordNetLemmatizer()
        stop_free = " ".join([i for i in tweetx.lower().split() if i not in stop_words])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        #final = re.sub(r' \1 ', s).split()
        abc.append(normalized)        
        #print tweetx
    return abc

X_train = data_cleaning(X_train)



In [28]:
vect = CountVectorizer()
tf_train = vect.fit_transform(X_train)
tf_val = vect.transform(X_val)



In [67]:
p = tf_train[y_train==1].sum(0) + 1
q = tf_train[y_train==0].sum(0) + 1
r = np.log((p/p.sum()) / (q/q.sum()))
b = np.log(len(p) / len(q))

In [78]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=0.1)
classifier.fit(tf_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(tf_val)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)
print ("Confusion Matrix:")
print(cm)

# Accuracy, Precision and Recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
score1 = accuracy_score(y_val,y_pred)
score2 = precision_score(y_val,y_pred)
score3= recall_score(y_val,y_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
[[1054  126]
 [  64  340]]


('Accuracy is ', 88.01, '%')
('Precision is ', 0.73)
('Recall is ', 0.84)


In [66]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=3.4, dual=True)
model.fit(tf_train, y_train)
preds = model.predict(tf_val)
accuracy = (preds == y_val).mean()
accuracy


0.87310606060606055

In [32]:

file = pd.DataFrame(zip(X_val,y_val,preds))
file.to_csv("check.csv")



In [34]:
from sklearn.metrics import confusion_matrix
cm =confusion_matrix(y_val, preds)
cm

array([[1119,   61],
       [ 151,  253]])